# CS 542 Fall 2025 Homework 4

Your task for this homework is to adapt this notebook to run on the Shared Compute Cluster (SCC).

## Background

This notebook builds a model detecting trees in images.
The data set consists of roughly 500 pictures and is currently stored in `/projectnb/ds542/materials/tree-or-not` on the SCC.
Most of them are from the Boston area, but some are from around the globe.
Most of them were taken outside, but some were taken inside or in more exotic locations.
Many other factors such as lighting, weather, and confounding bushes will make this a challenging problem.

## Instructions

1. Copy this notebook to your class directory on the SCC, i.e. `/projectnb/ds542/students/USERNAME`.
2. Run your copy notebook in Jupyter on the SCC using at least one GPU as explained in the SCC lecture.
3. Confirm that you have GPU access using the code below.
4. Modify the data loading code to read from the shared directory `/projectnb/ds542/materials/tree-or-not`.
5. Run the model training code. Make sure that it is running on GPU, not CPU.
6. Answer the question at the bottom of this notebook.
7. Submit just this notebook to Gradescope.

## Modules

Run but do not change this code.

In [ ]:
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import pandas as pd
import torch

## GPU Access

Run but do not change this code.

In [ ]:
def to_gpu(t):
    if torch.cuda.is_available():
        return t.cuda()
    return t

def to_numpy(t):
    return t.detach().cpu().numpy()

device = to_gpu(torch.ones(1,1)).device
device

## Load Data

This code originally relied on checking out or cloning [this repository](https://github.com/dl4ds/fa2024_midterm).

Modify this code to load from `/projectnb/ds542/materials/tree-or-not`.

In [ ]:
# the repository has images scaled to standard widths of 64, 128 and 256.
# you may use larger images if you prefer.

image_width = 64

def load_data_set(data_set_name):
    labels = pd.read_csv(f"fa2024_midterm/{data_set_name}.tsv", sep="\t")

    file_names = []
    images = []
    targets = []
    for i in range(labels.shape[0]):
        row = labels.iloc[i]
        try:
            image = imageio.imread(f"fa2024_midterm/images{image_width}/{row['filename']}")[...]
        except:
            print("SKIPPING ", row['filename'], "MISSING")
            continue

        if image.shape[0] != image.shape[1] * 3 // 4:
            print("SKIPPING ", row['filename'], image.shape)
            continue

        # convert from 0-255 to 0.0-1.0
        image = image / 255
        # prepend axis with length one
        # image = image.reshape(1, *image.shape)
        image = torch.tensor(image, device=device, dtype=torch.float32)
        # permute image dimensions to put color channel first
        image = torch.permute(image, [2, 0, 1])

        file_names.append(row['filename'])
        images.append(image)
        targets.append(row["target"])

    images = torch.stack(images)

    targets = torch.tensor(targets, device=device, dtype=torch.float32)
    targets = targets.long()

    return (file_names, images, targets)

train_data_set = load_data_set("train")
for t in train_data_set[1:]:
    print("TRAIN", t.shape, t.dtype, t.device)
(train_file_names, train_X, train_Y) = train_data_set

validation_data_set = load_data_set("validation")
for t in validation_data_set[1:]:
    print("VALIDATION", t.shape, t.dtype, t.device)
(validation_file_names, validation_X, validation_Y) = validation_data_set

In [ ]:
plt.imshow(to_numpy(torch.permute(train_X[0,:,:,:], (1, 2, 0))))

## Model Building

Do not modify any of this code.

In [ ]:
class TreeNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_0 = torch.nn.Conv2d(in_channels=3, out_channels=5, kernel_size=5, stride=2, device=device)
        self.conv_1 = torch.nn.Conv2d(in_channels=5, out_channels=5, kernel_size=3, stride=2, device=device)
        self.conv_2 = torch.nn.Conv2d(in_channels=5, out_channels=5, kernel_size=1, device=device)
        self.fc_3 = torch.nn.Linear(700, 2)

        self.relu = torch.nn.ReLU()

    def forward(self, X):

        X = self.conv_0(X)
        X = self.relu(X)

        X = self.conv_1(X)
        X = self.relu(X)

        X = self.conv_2(X)
        X = self.relu(X)

        # flatten channels and image dimensions
        X = X.reshape(X.shape[:-3] + (-1,))

        X = self.fc_3(X)

        return X

test_model = TreeNetwork().to(device)
test_output = test_model(train_X[:5,:,:,:])
assert test_output.shape == (5, 2)
del test_output

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
DEFAULT_EPOCHS = 1000 if torch.cuda.is_available() else 100

def train_model(model_class, epochs=DEFAULT_EPOCHS, learning_rate=1e-4, **kwargs):
    model = model_class(**kwargs)
    try:
        model = model.cuda()
    except:
        print("cuda() failed")
    model = torch.nn.DataParallel(model)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for i in range(epochs):
        model.train()

        optimizer.zero_grad(set_to_none=True)
        prediction = model(train_X)
        loss = loss_function(prediction, train_Y)
        loss.backward()
        optimizer.step()

        if (i + 1) % 50 == 0:
            liveloss_updates = {}
            with torch.no_grad():
                model.eval()

                metrics = {}
                def get_metrics(metrics_prefix, metrics_X, metrics_Y):
                    metrics_prediction = model(metrics_X)

                    return {
                        f"{metrics_prefix}loss": loss_function(metrics_prediction, metrics_Y)
                    }

                metrics.update(get_metrics("train_", train_X, train_Y))
                metrics.update(get_metrics("val_", validation_X, validation_Y))
                print("ITER", i+1, "METRICS", metrics)

    return model

In [ ]:
tree_model = train_model(TreeNetwork, epochs=2000)

## Question

Review the metrics printed above while training the model.
What trends do you notice in the training and validation losses?
Just state the trends that you see.
You do not need to explain them.

YOUR ANSWER HERE